In [ ]:
# |default_exp web_config

In [ ]:
# | hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# |hide
from nbdev.showdoc import *

# Manage web assets

In [ ]:
# | export

from pathlib import Path
from shutil import copy, copytree

from fastcore.all import *

from bedmap import utils
from bedmap.utils import get_version

In [ ]:
# |export


def get_bedmap_root() -> Path:
    # ipython doesn't have __file__ attribute
    if in_ipython():
        return Path(utils.__file__).parents[1]
    else:
        return Path(__file__).parents[1]

In [ ]:
# | export


def replace_web_strings(web_dir: Path, from_to: dict):
    """sequentially replace strings based on dict"""
    for i in ["index.html", "assets/js/tsne.js"]:
        f = web_dir / i
        t = f.read_text()
        for k, v in from_to.items():
            t = t.replace(k, v)
        f.write_text(t)

In [ ]:
# | export


def byo_logo(web_dir: Path, logo_path: Path):
    """copy logo into web folder and replace reference in index.html"""
    logo_dest = web_dir / "assets/images" / logo_path.name
    copy(logo_path, logo_dest)
    replace_web_strings(web_dir, {"dhlab-logo.svg": logo_path.name, "DHLab logo": "Custom logo"})

In [ ]:
# | export


def copy_web_assets(out_dir: str, tagline: str, logo: str) -> None:
    """Copy the /web directory from the bedmap source to the users cwd.
    Copies version number into assets.

    Args:
        out_dir (str): directory to copy web assets

    Returns:
        None
    """
    copy_root_dir = get_bedmap_root()
    src = copy_root_dir / "bedmap/web"

    # resolve will handle cases with ../ in the path
    dest = Path.cwd() / Path(out_dir).resolve()
    copytree(src.as_posix(), dest.as_posix(), dirs_exist_ok=True)

    # write version numbers into output
    replace_web_strings(dest, {"VERSION_NUMBER": get_version(), "Image Fields in a Local Collection": tagline})

    # replace logo, if bring-your-own provided
    if logo is not None:
        logo_path = Path.cwd() / Path(logo).resolve()
        byo_logo(dest, logo_path)

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()